In [ ]:
import logging
import pickle
import sys
import importlib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Machine Learning in...
~~5 Lines~~  ~~3 Lines~~ **ONE LINE** of Python!

In [ ]:
predictions = KMeans(6).fit_predict(StandardScaler().fit_transform(pd.read_csv('../data/processed/chr_final_2018.csv', index_col=0)))


In [ ]:
predictions

## Issues
This is problematic for many reasons, including:
* Code is not easy to read
* Analysis is not reproducible (how did we get that CSV file?)
* Have to retrain the model every time we want to make predictions
* No way of understanding the significance of the clusters
* No way of using the clusters in our business
* And where did that 6 come from??
* We also cheated by doing data ETL and cleaning outside of the notebook (which is most of the work according to the [80/20 rule](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/))

So lets redo that in a more comprehensive, comprehensible, reproducible way

# Data

## Load
* Download [raw county health data](http://www.countyhealthrankings.org/sites/default/files/chr_measures_CSV_2018.csv) from 
and save in data/raw

In [ ]:
# Raw data 
sdoh = pd.read_csv('../data/raw/chr_measures_CSV_2018.csv')

In [ ]:
sdoh.head()

* Run `python make_dataset.py` at command line from top level project directory to write to data/interim and data/processed
* See README for more details

In [ ]:
# Has all data, for reference (with mapped column names) before limiting columns and dropping na
# Written from create_dataset.py script
interim = pd.read_csv('../data/interim/chr_interim_2018.csv', index_col=0)

In [ ]:
interim.head()

In [ ]:
# Load final dataset
# Includes only columns used in the final model, will be fed directly into scaling and clustering
X = pd.read_csv('../data/processed/chr_final_2018.csv', index_col=0)

In [ ]:
X.head()

In [ ]:
len(interim)

In [ ]:
# 3000+ counties and county equivalents, some are dropped due to missing data
len(X)

In [ ]:
interim[X.columns].isnull().sum()

## Explore Data

In [ ]:
# Quick scatterplot matrix
sns.pairplot(X)

## Scale Data

In [ ]:
# Scale - zero mean and unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Returned object is a numpy array, so put columns and index back to use dataframe features
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

In [ ]:
# X_scaled

In [ ]:
# Can run some basic sanity checks - means should all be close to zero
X_scaled.mean()

In [ ]:
# Standard Deviations should all be one
X_scaled.std(ddof=0)  # divisor is N-ddof, default is sample mean meaning ddof=1

In [ ]:
# Can also check this automatically using assert - 

# Assert the absolute value of all means is less than machine epsilon (with factor of 10 wiggle room)
assert (X_scaled.mean().abs() < 1e-15).all(), 'Means should be zero post standardization'

In [ ]:
X_scaled.head()

# Clustering

## Select Number of Clusters
* Run k-means clustering for k=2 clusters to k=20 clusters  
* Select final number of clusters using the [elbow method](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set)
* We are using inertia (sum of squared distances to to closest cluster center) because sklearn includes it, can also use percent variance explained

In [ ]:
MIN=2
MAX=20
sum_squared_distances = []

for n_clusters in range(MIN,MAX):
    clustering = KMeans(n_clusters=n_clusters)
    clustering.fit(X_scaled)
    sum_squared_distances.append(clustering.inertia_)

In [ ]:
# Nice elbow at 6 clusters! Smaller one at 4
_ = plt.plot(range(MIN, MAX), sum_squared_distances)
_ = plt.xlabel('Number of Clusters k')
_ = plt.ylabel('Sum of Squared Errors')
# _ = plt.axvline(x=6, color='#696969', linestyle='--')

## Train final model

In [ ]:
# Note that the order of the clusters may be different each time you retrain the model -
# another reason saving the model is important
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Could use sklearn pipeline to put scaler and clustering into one step
clustering = KMeans(n_clusters=6)
clustering.fit_predict(X_scaled)

In [ ]:
# Now load previously trained model - cluster order will change every time
with open('../models/final_clustering.pkl', 'rb') as infile:
    reloaded_model = pickle.load(infile)
    
clustering = reloaded_model['kmeans']
scaler = reloaded_model['scaler']
label_map = reloaded_model['label_map']
    
X_scaled = scaler.fit_transform(X)

# reloaded_model

# Results

## Cluster Centers

In [ ]:
predictions = clustering.predict(X_scaled)

In [ ]:
predictions

In [ ]:
predictions = pd.Series(predictions, index=X.index)

In [ ]:
predictions.head(5)

In [ ]:
predictions.loc['Davidson,TN']

In [ ]:
# Percentage of counties in each cluster (note this is not percentage of the population!)
(100*predictions.value_counts().loc[range(0,6)]/len(predictions)).round(1)

In [ ]:
clustering.cluster_centers_

In [ ]:
clusters = pd.DataFrame(clustering.cluster_centers_,
                       columns=X.columns)

In [ ]:
clusters

In [ ]:
short_column_names = ['Income', 'Social Associations', 'Access to Exercise', 'Food Desert', 'Long Commute Alone', 'Percent Rural', 'Lack English Prof.']
unscaled_cluster_centers = pd.DataFrame(scaler.inverse_transform(clustering.cluster_centers_), 
                                        columns=short_column_names)  # X.columns)

In [ ]:
# Round for display
unscaled_cluster_centers.round(pd.Series([0,1,2,2,2,2,3], index=unscaled_cluster_centers.columns))

In [ ]:
ordered_names = ['Average Suburbia', 'Isolated Urbanites', 'Small Town USA', 
                 'Rural Challenges', 'Vulnerable Locations', 'Rural Relationships']
label_map = {}

for val in zip(ordered_names, range(len(ordered_names))):
    label_map[val[1]] = val[0]

In [ ]:
unscaled_cluster_centers.index = ordered_names

In [ ]:
unscaled_cluster_centers.round(pd.Series([0,1,2,2,2,2,3], index=unscaled_cluster_centers.columns))

## Visualization

### Seaborn Heatmap

In [ ]:
clusters.index = ordered_names
fig, ax = plt.subplots(figsize=(8,8))  
sns.heatmap(clusters, annot=unscaled_cluster_centers, ax=ax, cbar=False)

### Scatter Plot 
Using Principal Components To Project to 2D

In [ ]:
pca = PCA(n_components=2)
output = pd.DataFrame(pca.fit_transform(X_scaled), 
                      columns=['First Prinicpal Component', 'Second Principal Component'],
                     index=X.index)
output['cluster'] = predictions

In [ ]:
pca.explained_variance_ratio_

In [ ]:
components = pd.DataFrame(pca.components_, columns=X.columns)

In [ ]:
# First principal component
components[components.loc[0, :].abs().sort_values(ascending=False).index].loc[0, :]

In [ ]:
# Second principal component
components[components.loc[1, :].abs().sort_values(ascending=False).index].loc[1, :]

In [ ]:
output['Persona Name'] = output['cluster'].map(lambda x: ordered_names[x])

In [ ]:
output.head()

In [ ]:
output.loc['Davidson,TN']

In [ ]:
output.loc['Dickson,TN']

In [ ]:
# Uncomment height=7 line for nicer image with seaborn>=0.8
sns.set(font_scale=1.3)
lm = sns.lmplot(x='First Prinicpal Component', 
                y='Second Principal Component',
                data=output, #.sample(1000),
                hue='Persona Name',  # 'cluster'
                fit_reg=False, 
                height=7,
                scatter_kws={'s': 50},
                legend=True)
plt.title('Principle Component View of Data')

axes = lm.axes[0,0]

_ = axes.set_xlabel(r'More Urban + More Exercise Op. + Higher Income $\longrightarrow$')
_ = axes.set_ylabel(r'Better Commute + More Social Assoc. + Worse Food Desert $\longrightarrow$')

### Save serialized model

In [ ]:
final_clustering = {
    'scaler': scaler,
    'kmeans': clustering,
    'input_format': X.columns,
    'label_map': label_map,
    'notes': 'SDoH clustering model for Nashville Analytics Summit. Trained 7/2019 on 2018 County Health Ranking data'
}

In [ ]:
final_clustering.keys()

In [ ]:
final_clustering['label_map']

In [ ]:
with open('../models/example.pkl', 'wb') as outfile:
    pickle.dump(final_clustering, outfile)

In [ ]:
# Example reload
with open('../models/example.pkl', 'rb') as infile:
    reloaded_model = pickle.load(infile)

In [ ]:
reloaded_model['kmeans'].cluster_centers_

# Use custom module/package

In [ ]:
# analytics_manager = src.models.analytics_manager.AnalyticsManager('../models/final_clustering.pkl')


In [ ]:
!pwd

In [ ]:
sys.path.append('../')

In [ ]:
import src.models.analytics_manager

In [ ]:
# importlib.reload(src.models.analytics_manager)

In [ ]:
analytics_manager = src.models.analytics_manager.AnalyticsManager(
    '../models/final_clustering.pkl'
)

In [ ]:
analytics_manager.process()

In [ ]:
# analytics_manager.write_results()

In [ ]:
# AnalyticsManager()

In [ ]:
# %run ../src/models/analytics_manager.py

In [ ]:
# AnalyticsManager()

In [ ]:
# from src.models.analytics_manager import AnalyticsManager